In [ ]:
#! pip install pandas
#!pip install tensorflow

In [ ]:
# Import Libraries
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.layers import (
    BatchNormalization, Conv2D, MaxPooling2D, Flatten, Dropout, Dense
)
import matplotlib.pyplot as plt
import numpy as np
import os
import glob
from tqdm import tqdm
import cv2
import sklearn
import skimage
from skimage.transform import resize
import random
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import accuracy_score
import seaborn as sns
sns.set()

In [ ]:
#Diretório com o dataset utilizado para treinamento
train = "E:\Dados\Training and Validation"

#Diretório com o dataset utilizado para testes
test = "E:\Dados\Testing"


#Define que os dados devem ser carregados a partir de imagens originais
LOAD_FROM_IMAGES = True

# Lendo as imagens de um diretório e convertendo em arrays numéricos (x) 
# junto com as respectivas classes (y).
def get_data(folder):
    x = []
    y = []
    #Percorrendo o diretório 'folder'
    for folderName in os.listdir(folder):
        if not folderName.startswith("."):
            #Com base no nome do diretório,será definida a classe
            if folderName in ["nofire"]:
                label = 0
            elif folderName in ["fire"]:
                label = 1
            else:
                label = 2
            #Percorrendo as imagens dentro de cada classe
            for image_filename in tqdm(os.listdir(folder +"/" +folderName+"/")):
                new_size = (90, 90)
                img_arr = cv2.imread(folder + "/" + folderName + "/" + image_filename)
                if img_arr is not None:
                    #cada nova imagem é carregada e redimensionada para 128x128 pixels
                    img_arr = cv2.resize(img_arr, new_size)
                    # os arrays de imagem e classe são adicionados às listas 
                    x.append(img_arr)
                    y.append(label)
    x = np.asarray(x)
    y = np.asarray(y)
    return x,y

if LOAD_FROM_IMAGES:
    X_train,y_train = get_data(train)
    X_test, y_test = get_data(test)
    
    np.save("xtrain.npy",X_train)
    np.save("ytrain.npy",y_train)
    np.save("xtest.npy",X_test)
    np.save("ytest.npy",y_test)
else:
    X_train = np.load("xtrain.npy")
    y_train = np.load("ytrain.npy")
    X_test = np.load("xtest.npy")
    y_test = np.load("ytest.npy")

from sklearn.model_selection import train_test_split

# dividindo os dados de treinamento em dados de treinamento e validação. 
# Aqui está  divide em uma proporção de 80% para treinamento e 20% para validação.
X_train, X_valid, y_train, y_valid = train_test_split(X_train,y_train,test_size=0.2,shuffle=True)

# Normaliza os valores dos pixels das imagens dividindo-os por 255.0 
# para trazer todos os valores entre 0 e 1
X_train, X_valid, X_test = X_train / 255.0, X_valid / 255.0, X_test / 255.0

In [4]:
datagen = ImageDataGenerator(rotation_range=20, width_shift_range=0.2, height_shift_range=0.2, zoom_range=0.2, horizontal_flip=True)
datagen.fit(X_train)

In [5]:
model = models.Sequential()

# camada de convolução com 32 filtros, cada um com uma janela de convolução de 11x11. 
# Os filtros são aplicados com um passo de 4 pixels horizontalmente e 4 pixels verticalmente.
model.add(layers.Conv2D(32,(11,11),strides=(4, 4), input_shape=(90,90,3), use_bias=False))
# Adiciona uma camada de normalização em lotes após a camada de convolução para normalizar
# os valores de ativação da camada anterior
model.add(BatchNormalization())
# Aplica a função de ativação ReLU (Rectified Linear Unit) para introduzir não-linearidade.
model.add(layers.Activation('relu'))
# Adiciona uma camada de max pooling para reduzir a dimensão espacial dos mapas de 
# características resultantes. Usa uma janela de pooling de 3x3 e 
# um passo de 2 pixels horizontalmente e 2 pixels verticalmente
model.add(layers.MaxPooling2D((3,3), strides=(2,2)))
# Adiciona outra camada de convolução com 32 filtros, cada um com uma janela de 
# convolução de 5x5. A opção padding="same" mantém a saída da camada com a mesma 
# dimensão espacial da entrada.
model.add(layers.Conv2D(32,(5,5),padding="same", use_bias=False))
model.add(BatchNormalization())
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling2D((3,3), strides=(2,2)))
model.add(layers.MaxPooling2D((3,3), strides=(2,2)))

# Transforma as saídas 2D das camadas anteriores em um vetor 1D, 
# preparando os dados para as camadas totalmente conectadas (Dense).
model.add(layers.Flatten())

# Adiciona uma camada densamente conectada com 32 neurônios e função de ativação ReLU.
model.add(layers.Dense(128,activation="relu", use_bias=False))
# Adiciona uma camada de dropout com taxa de 50%, o que ajuda a evitar overfitting 
# ao desativar aleatoriamente metade das unidades de neurônios durante o treinamento.
model.add(Dropout(0.5))

model.add(layers.Dense(64,activation="relu", use_bias=False))
model.add(Dropout(0.5))

model.add(layers.Dense(1,activation='sigmoid', use_bias=False))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer = "adam" , loss = "binary_crossentropy", metrics=["accuracy"])

from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(
    monitor="accuracy",
    patience=50,
    restore_best_weights=True)

batch_size=16
epochs=300

#history =  model.fit(X_train,y_train,validation_data=(X_valid,y_valid),batch_size=batch_size, epochs=epochs,verbose=1,callbacks=[early_stopping])
history = model.fit(datagen.flow(X_train, y_train),validation_data=(X_valid, y_valid),batch_size=batch_size,epochs=epochs, verbose=1, callbacks=[early_stopping])

In [ ]:
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)

print('Test loss:', round(score[0],4))
print('Test accuracy:', round(score[1],4))

In [ ]:
y_test_pred = model.predict(X_test)

y_pred = (y_test_pred > 0.5)

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

y_pred.shape

y_test.shape

print(classification_report(y_test, y_pred)) 

In [12]:
model.save('fire_forest_cnn.h5')

In [ ]:
conf_matrix = confusion_matrix(y_test, y_pred)
print(conf_matrix)